In [ ]:
%pip install ipywidgets matplotlib rasterio

In [6]:
import os
import pandas as pd
import rasterio
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
import ipywidgets as widgets

In [ ]:
# Folder with your .tif images
image_dir = "square_chips"  # change this to your folder path
output_csv = "cafo_labels.csv"

# Load existing labels if any
if os.path.exists(output_csv):
    df = pd.read_csv(output_csv)
    labeled = set(df['filename'])
else:
    df = pd.DataFrame(columns=['filename', 'label'])
    labeled = set()

# List of tif images that need labeling
image_files = sorted([f for f in os.listdir(image_dir) if f.endswith('.tif') and f not in labeled])

def show_tif(image_path, title):
    with rasterio.open(image_path) as src:
        img = src.read([1, 2, 3])  # read RGB bands
        img = np.transpose(img, (1, 2, 0))  # bands first → last
        # Normalize for display if needed
        img = img / 255.0
        plt.imshow(img)
        plt.title(title)
        plt.axis('off')
        plt.show()

def label_image(index=0):
    if index >= len(image_files):
        clear_output()
        print("✅ All images labeled.")
        return

    filename = image_files[index]
    image_path = os.path.join(image_dir, filename)

    clear_output(wait=True)
    print(f"Labeling {index + 1}/{len(image_files)}: {filename}")

    show_tif(image_path, filename)

    def on_label(chosen_label):
        nonlocal index
        df.loc[len(df)] = [filename, chosen_label]
        df.to_csv(output_csv, index=False)
        index += 1
        label_image(index)

    buttons = []
    for label in ['Wet', 'Dry', 'Unknown']:
        btn = widgets.Button(description=label, layout=widgets.Layout(width='100px'))
        btn.on_click(lambda b, label=label: on_label(label))
        buttons.append(btn)

    display(widgets.HBox(buttons))

label_image()